# Imports

In [2]:
import pandas as pd
import io, requests, os 
from bs4 import BeautifulSoup
from tqdm import tqdm
import numpy as np
pd.set_option('display.max_columns', None)

output_folder = 'CovidOct25-2022' # write the cleaned processed files here
support_folder = 'Support files' # cache the raw source files here for future usage

if not os.path.exists(output_folder):
    os.makedirs(output_folder, exist_ok=True)

if not os.path.exists(support_folder):
    os.makedirs(support_folder, exist_ok=True)

# Source files

## Population Estimation

PVI dashboard has age distribution data from 2018. So we updated that with [this age distribution data](https://www.census.gov/programs-surveys/popest/technical-documentation/research/evaluation-estimates/2020-evaluation-estimates/2010s-county-detail.html) from 2020. The data codebook is [here](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2020/cc-est2020-agesex.pdf).

In [2]:
url = 'https://www2.census.gov/programs-surveys/popest/datasets/2010-2020/counties/asrh/CC-EST2020-AGESEX-ALL.csv'
response = requests.get(url)
file_object = io.StringIO(response.content.decode('latin1')) # contains letters undecodable for utf-8
df = pd.read_csv(file_object, low_memory=False) # low_memory=False because features have mixed types

# in case the download by code is not working
# df = pd.read_csv(f'{output_folder}/CC-EST2020-AGESEX-ALL.csv', encoding='latin1', low_memory=False)
df.head(3)

,SUMLEV,STATE,COUNTY,STNAME,CTYNAME,YEAR,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
0,50,1,1,Alabama,Autauga County,1,54571,26569,28002,3579,1866,1713,7418,3747,3671,3616,1842,1774,4617,2335,2282,41804,20046,21758,39958,19114,20844,22100,10867,11233,14730,7115,7615,14065,6843,7222,6546,2821,3725,3579,1866,1713,3991,2001,1990,4290,2171,2119,4290,2213,2077,3080,1539,1541,3157,1543,1614,3330,1594,1736,4157,2004,2153,4086,1974,2112,4332,2174,2158,3873,1866,2007,3083,1524,1559,2777,1279,1498,2277,1014,1263,1736,807,929,1251,546,705,731,295,436,551,159,392,37,35.9,37.9
1,50,1,1,Alabama,Autauga County,2,54582,26576,28006,3582,1868,1714,7425,3752,3673,3617,1842,1775,4617,2335,2282,41804,20045,21759,39958,19114,20844,22103,10867,11236,14732,7115,7617,14067,6845,7222,6542,2819,3723,3582,1868,1714,3994,2004,1990,4294,2173,2121,4291,2213,2078,3080,1539,1541,3160,1545,1615,3329,1593,1736,4157,2004,2153,4086,1973,2113,4333,2176,2157,3874,1866,2008,3083,1524,1559,2777,1279,1498,2277,1014,1263,1734,806,928,1250,546,704,730,294,436,551,159,392,37,35.9,37.8
2,50,1,1,Alabama,Autauga County,3,54761,26667,28094,3575,1862,1713,7400,3720,3680,3565,1819,1746,4670,2362,2308,42038,20190,21848,40221,19266,20955,22194,10939,11255,14815,7185,7630,14137,6872,7265,6599,2847,3752,3575,1862,1713,3964,1984,1980,4292,2163,2129,4228,2181,2047,3151,1573,1578,3188,1573,1615,3369,1614,1755,4142,2000,2142,4116,1998,2118,4321,2163,2158,3908,1884,2024,3119,1533,1586,2789,1292,1497,2289,1018,1271,1752,814,938,1259,552,707,743,299,444,556,164,392,37.1,36,37.9


In [3]:
df = df[df['YEAR']==13] # category 13 is for year 2020 estimation
df.drop(columns=['SUMLEV', 'YEAR'], inplace=True)
df['Name'] = df['STNAME'] + ', ' + df['CTYNAME'].apply(lambda x: x.replace(' County', ''))
df['FIPS'] = df['STATE'] * 1000 + df['COUNTY']

identity_columns = ['STATE','COUNTY','FIPS','STNAME','CTYNAME','Name']
other_columns = [col for col in df.columns if col not in identity_columns]
df = df[identity_columns+other_columns]
df.head(3)

,STATE,COUNTY,FIPS,STNAME,CTYNAME,Name,POPESTIMATE,POPEST_MALE,POPEST_FEM,UNDER5_TOT,UNDER5_MALE,UNDER5_FEM,AGE513_TOT,AGE513_MALE,AGE513_FEM,AGE1417_TOT,AGE1417_MALE,AGE1417_FEM,AGE1824_TOT,AGE1824_MALE,AGE1824_FEM,AGE16PLUS_TOT,AGE16PLUS_MALE,AGE16PLUS_FEM,AGE18PLUS_TOT,AGE18PLUS_MALE,AGE18PLUS_FEM,AGE1544_TOT,AGE1544_MALE,AGE1544_FEM,AGE2544_TOT,AGE2544_MALE,AGE2544_FEM,AGE4564_TOT,AGE4564_MALE,AGE4564_FEM,AGE65PLUS_TOT,AGE65PLUS_MALE,AGE65PLUS_FEM,AGE04_TOT,AGE04_MALE,AGE04_FEM,AGE59_TOT,AGE59_MALE,AGE59_FEM,AGE1014_TOT,AGE1014_MALE,AGE1014_FEM,AGE1519_TOT,AGE1519_MALE,AGE1519_FEM,AGE2024_TOT,AGE2024_MALE,AGE2024_FEM,AGE2529_TOT,AGE2529_MALE,AGE2529_FEM,AGE3034_TOT,AGE3034_MALE,AGE3034_FEM,AGE3539_TOT,AGE3539_MALE,AGE3539_FEM,AGE4044_TOT,AGE4044_MALE,AGE4044_FEM,AGE4549_TOT,AGE4549_MALE,AGE4549_FEM,AGE5054_TOT,AGE5054_MALE,AGE5054_FEM,AGE5559_TOT,AGE5559_MALE,AGE5559_FEM,AGE6064_TOT,AGE6064_MALE,AGE6064_FEM,AGE6569_TOT,AGE6569_MALE,AGE6569_FEM,AGE7074_TOT,AGE7074_MALE,AGE7074_FEM,AGE7579_TOT,AGE7579_MALE,AGE7579_FEM,AGE8084_TOT,AGE8084_MALE,AGE8084_FEM,AGE85PLUS_TOT,AGE85PLUS_MALE,AGE85PLUS_FEM,MEDIAN_AGE_TOT,MEDIAN_AGE_MALE,MEDIAN_AGE_FEM
12,1,1,1001,Alabama,Autauga County,"Alabama, Autauga",56130,27231,28899,3346,1739,1607,6566,3366,3200,3114,1553,1561,4416,2267,2149,44657,21353,23304,43104,20573,22531,21567,10606,10961,14798,7166,7632,14848,7161,7687,9042,3979,5063,3346,1739,1607,3494,1785,1709,3833,1961,1872,3622,1825,1797,3147,1615,1532,3822,1893,1929,3636,1756,1880,3743,1818,1925,3597,1699,1898,3820,1844,1976,3675,1811,1864,3912,1915,1997,3441,1591,1850,2790,1351,1439,2386,1032,1354,1796,749,1047,1142,494,648,928,353,575,39.3,37.8,40.5
26,1,3,1003,Alabama,Baldwin County,"Alabama, Baldwin",227989,110420,117569,12092,6319,5773,24636,12572,12064,11536,5787,5749,16067,8055,8012,185376,88563,96813,179725,85742,93983,77312,38054,39258,52651,25693,26958,62077,29384,32693,48930,22610,26320,12092,6319,5773,13356,6874,6482,14222,7179,7043,13371,6684,6687,11290,5677,5613,12517,6254,6263,12865,6256,6609,13524,6579,6945,13745,6604,7141,14328,6941,7387,14519,6963,7556,16420,7756,8664,16810,7724,9086,15519,7146,8373,13806,6532,7274,9457,4503,4954,5607,2554,3053,4541,1875,2666,43.8,42.5,45.2
40,1,5,1005,Alabama,Barbour County,"Alabama, Barbour",24652,13018,11634,1315,673,642,2574,1275,1299,1164,593,571,1992,1162,830,20183,10777,9406,19599,10477,9122,9300,5525,3775,6441,3917,2524,6188,3234,2954,4978,2164,2814,1315,673,642,1351,659,692,1520,763,757,1394,726,668,1465,882,583,1762,1113,649,1648,1004,644,1559,899,660,1472,901,571,1552,869,683,1467,795,672,1659,827,832,1510,743,767,1499,689,810,1406,619,787,971,439,532,587,245,342,515,172,343,40.9,38.9,44.5


In [4]:
# https://www.census.gov/programs-surveys/geography/technical-documentation/county-changes/2010.html
if df[df['FIPS'].isin([2063, 2066])].shape[0] == 2:
    values = df[df['FIPS']==2063][other_columns].values[0] + df[df['FIPS']==2066][other_columns].values[0]
    df.loc[len(df.index)] = [2, 261, 2261, 'Alaska', 'Valdez-Cordova', 'Valdez-Cordova County'] + list(values)
    df = df[~df['FIPS'].isin([2063, 2066])]

    df = df.sort_values(by=['FIPS'])

In [5]:
df.to_csv(f'{support_folder}/Population.csv', index=False)

## Sovial Vulnerability Index
If the downloading bellow doesn't work, just directly go to [this link](https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html), select geography type counties for United States and download the csv file for year 2020. Then load that csv file.

In [20]:
url = 'https://svi.cdc.gov/Documents/Data/2020_SVI_Data/CSV/SVI2020_US_COUNTY.csv'
response = requests.get(url)
file_object = io.StringIO(response.content.decode('utf-8'))

svi_filename = url.split('/')[-1]
df = pd.read_csv(file_object, low_memory=False)
df.to_csv(f"{support_folder}/{svi_filename}", index=False)

# Feature processing

## Age distribution

In [7]:
df = pd.read_csv(f'{support_folder}/Population.csv')
df['AgeDist'] = df['AGE65PLUS_TOT'].astype(int) / df['POPESTIMATE'].astype(int)
# PVI data came min max scaled
df = df[['FIPS', 'AgeDist']]
df.head(3)

,FIPS,AgeDist
0,1001,0.161090
1,1003,0.214616
2,1005,0.201931


In [8]:
df.round(4).to_csv(f'{output_folder}/Age Distribution_.csv', index=False)

## Health disparity

Corresponding column names in the SVI file is mentioned in ().

| Component(s)	| Update Freq.	| Description/Rationale | Source |
|:---:|:---:|:---:|:---:|
| Uninsured|	Static	|	Percentage uninsured in the total civilian noninstitutionalized population estimate (EP_UNINSUR). Individuals without insurance are more likely to be undercounted in infection statistics, and may have more severe outcomes due to lack of treatment.|	[2020 SVI](https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html)|
| Socioeconomic Status|	Static	|	Integrates data on estimated percent below poverty (EPL_POV150), percent civilian (age 16+) unemployed (historical) (EPL_UNEMP), housing cost-burdened occupied housing units (EPL_HBURD), income (EPL_PCI), and percent without a high school diploma (EPL_NOHSDP). Lower SES are more likely to be undercounted in infection statistics, and may have more severe outcomes due to lack of treatment.|	[2020 SVI](https://www.atsdr.cdc.gov/placeandhealth/svi/data_documentation_download.html)|

In [24]:
df = pd.read_csv(f"{support_folder}/{svi_filename}")
df.head(3)

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,E_HU,M_HU,E_HH,M_HH,E_POV150,M_POV150,E_UNEMP,M_UNEMP,E_HBURD,M_HBURD,E_NOHSDP,M_NOHSDP,E_UNINSUR,M_UNINSUR,E_AGE65,M_AGE65,E_AGE17,M_AGE17,E_DISABL,M_DISABL,E_SNGPNT,M_SNGPNT,E_LIMENG,M_LIMENG,E_MINRTY,M_MINRTY,E_MUNIT,M_MUNIT,E_MOBILE,M_MOBILE,E_CROWD,M_CROWD,E_NOVEH,M_NOVEH,E_GROUPQ,M_GROUPQ,EP_POV150,MP_POV150,EP_UNEMP,MP_UNEMP,EP_HBURD,MP_HBURD,EP_NOHSDP,MP_NOHSDP,EP_UNINSUR,MP_UNINSUR,EP_AGE65,MP_AGE65,EP_AGE17,MP_AGE17,EP_DISABL,MP_DISABL,EP_SNGPNT,MP_SNGPNT,EP_LIMENG,MP_LIMENG,EP_MINRTY,MP_MINRTY,EP_MUNIT,MP_MUNIT,EP_MOBILE,MP_MOBILE,EP_CROWD,MP_CROWD,EP_NOVEH,MP_NOVEH,EP_GROUPQ,MP_GROUPQ,EPL_POV150,EPL_UNEMP,EPL_HBURD,EPL_NOHSDP,EPL_UNINSUR,SPL_THEME1,RPL_THEME1,EPL_AGE65,EPL_AGE17,EPL_DISABL,EPL_SNGPNT,EPL_LIMENG,SPL_THEME2,RPL_THEME2,EPL_MINRTY,SPL_THEME3,RPL_THEME3,EPL_MUNIT,EPL_MOBILE,EPL_CROWD,EPL_NOVEH,EPL_GROUPQ,SPL_THEME4,RPL_THEME4,SPL_THEMES,RPL_THEMES,F_POV150,F_UNEMP,F_HBURD,F_NOHSDP,F_UNINSUR,F_THEME1,F_AGE65,F_AGE17,F_DISABL,F_SNGPNT,F_LIMENG,F_THEME2,F_MINRTY,F_THEME3,F_MUNIT,F_MOBILE,F_CROWD,F_NOVEH,F_GROUPQ,F_THEME4,F_TOTAL,E_DAYPOP,E_NOINT,M_NOINT,E_AFAM,M_AFAM,E_HISP,M_HISP,E_ASIAN,M_ASIAN,E_AIAN,M_AIAN,E_NHPI,M_NHPI,E_TWOMORE,M_TWOMORE,E_OTHERRACE,M_OTHERRACE,EP_NOINT,MP_NOINT,EP_AFAM,MP_AFAM,EP_HISP,MP_HISP,EP_ASIAN,MP_ASIAN,EP_AIAN,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE
0,1,Alabama,AL,1001,Autauga,1001,"Autauga County, Alabama",594.455838,55639,0,23697,68,21559,366,12611,1349,736,185,5029,576,4273,562,4345,725,8490,85,13143,49,9658,856,1608,302,363,229,14479,578,918,292,4313,440,339,162,1167,320,578,157,22.9,2.4,2.9,0.7,23.3,2.6,11.3,1.5,8.0,1.3,15.3,0.2,23.6,0.1,17.7,1.6,7.5,1.4,0.7,0.4,26.0,1.0,3.9,1.2,18.2,1.8,1.6,0.8,5.4,1.5,1.0,0.3,0.4624,0.1397,0.7078,0.5080,0.4586,2.2765,0.4045,0.1827,0.6986,0.6830,0.7922,0.4736,2.8301,0.7362,0.6337,0.6337,0.6337,0.6050,0.7486,0.4023,0.4764,0.1569,2.3892,0.4309,8.1295,0.5325,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,41810,7100,913,10849,345,1601,0,649,174,155,102,0,29,1124,374,101,101,12.9,0.0,19.5,0.6,2.9,0.0,1.2,0.3,0.3,0.2,0.0,0.1,2.0,0.7,0.2,0.3
1,1,Alabama,AL,1003,Baldwin,1003,"Baldwin County, Alabama",1589.835294,218289,0,116747,180,84047,1143,36413,2802,4027,714,19350,1526,14823,1344,20501,1932,44716,109,46993,0,30615,1565,3317,547,1593,486,37334,1866,19513,979,11893,868,1280,364,2627,397,2954,403,16.9,1.3,3.9,0.7,23.0,1.8,9.5,0.9,9.5,0.9,20.5,0.1,21.5,0.0,14.2,0.7,3.9,0.6,0.8,0.2,17.1,0.9,16.7,0.8,10.2,0.7,1.5,0.4,3.1,0.5,1.4,0.2,0.1770,0.2861,0.8912,0.3647,0.5767,2.2957,0.4112,0.6496,0.4077,0.3689,0.1738,0.5150,2.1150,0.2724,0.5022,0.5022,0.5022,0.9574,0.4885,0.3600,0.1257,0.3157,2.2473,0.3612,7.1602,0.3552,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,218607,24453,2477,19027,744,9947,0,2033,352,1327,371,10,16,4250,871,740,740,11.4,0.0,8.7,0.3,4.6,0.0,0.9,0.2,0.6,0.2,0.0,0.1,1.9,0.4,0.3,0.2
2,1,Alabama,AL,1005,Barbour,1005,"Barbour County, Alabama",885.007618,25026,0,12057,119,9322,338,8965,673,649,182,2305,298,4497,387,2362,311,4777,16,5222,29,4159,287,1029,217,433,205,13694,271,170,77,3644,347,350,192,1039,202,2910,264,40.6,3.0,6.9,1.9,24.7,3.1,25.3,2.2,10.7,1.4,19.1,0.1,20.9,0.1,18.8,1.4,11.0,2.3,1.8,0.9,54.7,1.1,1.4,0.6,30.2,2.8,3.8,2.1,11.1,2.1,11.6,1.1,0.9586,0.8071,0.5121,0.9688,0.6680,3.9146,0.9532,0.5216,0.3316,0.7581,0.9656,0.7460,3.3229,0.9453,0.8962,0.8962,0.8962,0.2565,0.9427,0.8549,0.9456,0.9513,3.9510,0.9949,12.0847,0.9927,1,0,0,1,0,2,0,0,0,1,0,1,0,0,0,1,0,1,1,3,6,27133,6249,675,11889,179,1110,0,122,25,81,63,1,2,334,149,157,157,28.3,0.3,47.5,0.7,4.4,0.0,0.5,0.1,0.3,0.3,0.0,0.1,1.3,0.6,0.6,0.4


In [25]:
df['FIPS'].dtypes

dtype('int64')

In [28]:
df = df[df['RPL_THEME1'] >= 0]
feature_column = 'HealthDisp'
df[feature_column] = df[['RPL_THEME1', 'EP_UNINSUR']].apply(np.mean, axis=1)
df = df[['FIPS', feature_column]]

# https://www.census.gov/programs-surveys/geography/technical-documentation/county-changes/2010.html
if df[df['FIPS'].isin([2063, 2066])].shape[0] == 2:
    values = df[df['FIPS']==2063][feature_column].values[0] + df[df['FIPS']==2066][feature_column].values[0]
    
    df.loc[len(df.index)] = [2261, values/2]
    df = df[~df['FIPS'].isin([2063, 2066])]

    df = df.sort_values(by=['FIPS'])

df['FIPS'] = df['FIPS'].astype(int)
df.round(3).to_csv(f'{output_folder}/Health Disparities_.csv', index=False)

### Additional processing for TFT1 notebook
This is done since the TFT1 notebook can not handle normal static files and all files need to be in dynamic format for it

In [12]:
def convert_to_dynamic(df, feature_column, start, end):
    total_days = (end - start).days + 1

    values = df[feature_column].values
    # min max scaling
    # df[feature_column] = (values - values.min())/(values.max() - values.min())

    dates = [start + pd.to_timedelta(day, unit='D') for day in range(total_days)]
    values = df[feature_column].values

    additional = pd.DataFrame({date:values for date in dates}, columns=dates)
    df = pd.concat([df, additional],  axis=1)
    df.drop(columns=[feature_column], inplace=True)
    return df

In [29]:
start = pd.to_datetime('2020-03-01')
end = pd.to_datetime('2022-10-25')

map = [('Health Disparities', 'HealthDisp'), ('Age Distribution', 'AgeDist')]
for value in map:
    filename, feature_column = value[0], value[1]

    df = pd.read_csv(f'{output_folder}/{filename}_.csv')
    df = convert_to_dynamic(df, feature_column, start, end)
    df.round(4).to_csv(f'{output_folder}/{filename}.csv', index=False)